## Importing Packages

In [1]:
import xarray as xr
import pandas as pd
import numpy  as np
import scipy

import netCDF4
from netCDF4 import Dataset

import cartopy.crs as ccrs
import cartopy
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib

import sys
import subprocess

import seaborn as sns

import datetime

import glob
import os

import warnings
warnings.filterwarnings("ignore")

## Dataset

#### Map Information

In [2]:
shp = 'd:/00_Masters/01_Dados/Shapes/brasil_UF.shp'

extent = [-39.1,-33, -14.3,-4.3] # lonmin lonmax latmin latmax
proj = ccrs.PlateCarree()

#### Glorys Dataset

In [3]:
# # Get a list of all .nc files available in different folders
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
dsmerged = xr.open_mfdataset(filenames)

# # Corrige erro de datas duplicadas pelo mfdataset
_, index = np.unique(dsmerged['time'], return_index=True)
dsmerged = dsmerged.isel(time=index)

             Defining DEPTHS of interest 

In [4]:
Depth_0 = dsmerged.sel(depth=0,method='nearest')
temp_0 = Depth_0['thetao']
clim_0 = temp_0.mean('time')

Depth_30 = dsmerged.sel(depth=30,method='nearest')
temp_30 = Depth_30['thetao']
clim_30 = temp_30.mean('time')


depths = [0,30]

# Anomalias

### Monthly Climatology

In [17]:
M_0 = temp_0.groupby("time.month").mean()
Anom_0 = M_0 - clim_0
anom_std_0 = Anom_0.std(dim='month')
M_30 = temp_30.groupby("time.month").mean()
Anom_30 = M_30 - clim_30
anom_std_30 = Anom_30.std(dim='month')

M_Anom = [Anom_0,Anom_30]

Months = [1,2,3,4,5,6,7,8,9,10,11,12]

##### File - Max & Min

In [29]:
Months_names = ['January','February','March',
                'April','May','June',
                'July','August','September',
                'October','November','December']

for n,d in enumerate(depths):
    f = open("D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_temp/Outputs/Limite_Mensal_Anomalias_" + str(d) + ".txt", "w+")
    for i,m in enumerate(Months):
        print('Processing: ' + Months_names[i])
        cc = M_Anom[n].sel(month=m)
        vmin = cc.min().values
        vmax = cc.max().values
        print('\n' + Months_names[i] + ':', file=f)
        print('Min.Temp =', vmin, file=f)
        print('Min.Temp =', vmax, file=f)
        print('Done: '+ Months_names[i])
    
    f.close()  
    print('\nFile Closed')
    if n != np.size(depths):
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')

Processing: January
Done: January
Processing: February
Done: February
Processing: March
Done: March
Processing: April
Done: April
Processing: May
Done: May
Processing: June
Done: June
Processing: July
Done: July
Processing: August
Done: August
Processing: September
Done: September
Processing: October
Done: October
Processing: November
Done: November
Processing: December
Done: December

File Closed

Starting Next Depth

Processing: January
Done: January
Processing: February
Done: February
Processing: March
Done: March
Processing: April
Done: April
Processing: May
Done: May
Processing: June
Done: June
Processing: July
Done: July
Processing: August
Done: August
Processing: September
Done: September
Processing: October
Done: October
Processing: November
Done: November
Processing: December
Done: December

File Closed

Starting Next Depth



##### Figure - All maps in ONE

In [19]:
Months_names = ['JAN','FEB','MAR',
                'APR','MAY','JUN',
                'JUL','AUG','SEP',
                'OCT','NOV','DEC']
                
## IMPORTANTE FIXAR VALORES PARA OS MINIMOS E MAXIMOS ##
Tmin = [-1,-1]
Tmax = [1,1]

Versao = 0

for i,d in enumerate(depths):

    fig = plt.figure(figsize=(16.53,11.69), dpi = 300, tight_layout=True) # open a new figure window and set the resolution
    fig.suptitle('Climatology\nDepth = '+str(round(float(M_Anom[i]['depth'].values),2))+'m', fontsize=16, y=0.73)
    fig.subplots_adjust(left=None, bottom=0.1, right=0.835, top=.7, wspace=0.1, hspace=0.005)
    fig.patch.set_facecolor('white')

    for n,m in enumerate(Months):
        if n>0:
            print('Done: '+ Months_names[n-1])

        print('Processing: ' + Months_names[n])

        cc = M_Anom[i].sel(month=m)
        del cc['month']
        del cc['depth']

        ax = plt.subplot(2, 6, n+1, projection=proj)
        ax.set_extent(extent)
        ax.text(0.9, 0.02, Months_names[n], fontweight='bold', fontsize=12, transform=ax.transAxes, ha="right",)


        gl = ax.gridlines(crs=proj, draw_labels=True,
                          linewidth=1, color='gray', alpha=0.5, linestyle='--')
        gl.xlocator = mticker.FixedLocator(np.arange(-34,-39.5,-2))
        gl.ylocator = mticker.FixedLocator(np.arange(-5,-14.5,-2.5)) 
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        plt.rcParams.update({'font.size': 10})

        if n != 0 and n != 6:
            gl.ylabels_left = False

        if n < 6:
            gl.xlabels_bottom = False


        shape_feature = ShapelyFeature(Reader(shp).geometries(),
                                        ccrs.PlateCarree(), edgecolor='gray',linewidth=.5)
        ax.add_feature(shape_feature,facecolor='oldlace')

        im = cc.plot(ax=ax, transform=proj, vmin=Tmin[i], vmax=Tmax[i],
                     cmap='RdBu_r', add_colorbar=False,)

    cbar_ax = fig.add_axes([0.85, 0.125, 0.01, 0.55])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation="vertical",extend='both') #shrink=.8
    cbar.set_label(label='Anomaly (°C)', size=12)
    cbar.ax.tick_params(labelsize=12)

    print('Done: '+ Months_names[n])

    plt.savefig('D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_temp/Figuras/Temp-Anom_d'+str(d)+'_v'+ str(Versao) + '.png',
                bbox_inches='tight') # Save image as PNG file without large white borders

    print('Picture: Saved')
    plt.close()
    if i < np.size(depths)-1:
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')
    plt.show()

Processing: JAN
Done: JAN
Processing: FEB
Done: FEB
Processing: MAR
Done: MAR
Processing: APR
Done: APR
Processing: MAY
Done: MAY
Processing: JUN
Done: JUN
Processing: JUL
Done: JUL
Processing: AUG
Done: AUG
Processing: SEP
Done: SEP
Processing: OCT
Done: OCT
Processing: NOV
Done: NOV
Processing: DEC
Done: DEC
Picture: Saved

Starting Next Depth

Processing: JAN
Done: JAN
Processing: FEB
Done: FEB
Processing: MAR
Done: MAR
Processing: APR
Done: APR
Processing: MAY
Done: MAY
Processing: JUN
Done: JUN
Processing: JUL
Done: JUL
Processing: AUG
Done: AUG
Processing: SEP
Done: SEP
Processing: OCT
Done: OCT
Processing: NOV
Done: NOV
Processing: DEC
Done: DEC
Picture: Saved

Done


### Seasonal Climatology

In [5]:
Season = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]] 
Season_names = ['SUMMER','AUTUMN','WINTER','SPRING']

Season_climatology_0 = {}
Season_climatology_30 = {}
S_Anom_0 = {}
S_Anom_30 = {}
for s,sn in zip(Season,Season_names):
    Season_climatology_0[sn] = temp_0.sel(time = np.in1d(temp_0['time.month'], s)).mean('time')
    S_Anom_0[sn] = Season_climatology_0[sn] - clim_0
    Season_climatology_30[sn] = temp_30.sel(time = np.in1d(temp_30['time.month'], s)).mean('time')
    S_Anom_30[sn] = Season_climatology_30[sn] - clim_30

S_Anom = [S_Anom_0,S_Anom_30]

##### File - Max & Min

In [34]:
for n,d in enumerate(depths):
    f = open("D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_temp/Outputs/Limites_Sazonal_Anomalia" + str(d) + ".txt", "w+")
    for i,sn in enumerate(Season_names):
        print('Processing: ' + Season_names[i])
        cc = S_Anom[n][sn]
        vmin = cc.min().values
        vmax = cc.max().values
        print('\n' + Season_names[i] + ':', file=f)
        print('Min.Temp =', vmin, file=f)
        print('Min.Temp =', vmax, file=f)
        print('Done: '+ Season_names[i])
    
    f.close()  
    print('\nFile Closed')
    if n < np.size(depths):
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING

File Closed

Starting Next Depth

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING

File Closed

Starting Next Depth



##### Figure - All maps in ONE

In [6]:
Tmin = [-1,-1]
Tmax = [1,1]
## IMPORTANTE FIXAR VALORES PARA OS MINIMOS E MAXIMOS ##

Versao = 0

for i,d in enumerate(depths):

    fig = plt.figure(figsize=(3.4,6.4),dpi=300)
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.85, top=0.8, wspace=0.05, hspace=0.05)
    fig.suptitle('Climatology\nDepth = '+str(round(float(S_Anom[i]['SUMMER']['depth'].values),2))+'m', fontsize=12, y=0.85)
    fig.patch.set_facecolor('white')

    for n,s in enumerate(Season):
        if n > 0:
            print('Done: '+ Season_names[n-1])

        print('Processing: ' + Season_names[n])

        cS = S_Anom[i][Season_names[n]]
        del cS['depth']

        ax = plt.subplot(2, 2, n+1, projection=proj)
        ax.set_extent(extent)
        ax.text(0.9, 0.02, Season_names[n], fontweight='bold', fontsize=10, transform=ax.transAxes, ha="right",)

        gl = ax.gridlines(crs=proj, draw_labels=True,
                          linewidth=1, color='gray', alpha=0.5, linestyle='--')
        gl.xlocator = mticker.FixedLocator(np.arange(-34,-39.5,-2))
        gl.ylocator = mticker.FixedLocator(np.arange(-5,-14.5,-2.5)) 
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        plt.rcParams.update({'font.size': 8})

        if n != 0 and n != 2:
            gl.ylabels_left = False

        if n < 2:
            gl.xlabels_bottom = False

        shape_feature = ShapelyFeature(Reader(shp).geometries(),
                                        ccrs.PlateCarree(), edgecolor='gray',linewidth=.5)
        ax.add_feature(shape_feature,facecolor='oldlace')

        im = cS.plot(ax=ax, transform=proj, vmin=Tmin[i], vmax=Tmax[i],
                     cmap='RdBu_r', add_colorbar=False,)

    print('Done: '+ Season_names[n])   

    cbar_ax = fig.add_axes([0.90, 0.115, 0.02, 0.67])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation="vertical",extend='both')
    cbar.set_label(label='Temp. (°C)', size=8)
    cbar.ax.tick_params(labelsize=8)

    plt.savefig('D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_temp/Figuras/Temp-Anom_Sazonal_d'+str(d)+'_v'+ str(Versao) + '.png',
                bbox_inches='tight')

    print('Picture: Saved')
    plt.close()
    if i < np.size(depths)-1:
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')
    plt.show()

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING
Picture: Saved

Starting Next Depth

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING
Picture: Saved

Done


-----